In [33]:
from datetime import datetime
import requests
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.exc import SQLAlchemyError
import time

In [34]:
# set up
Base = declarative_base()

class ExchangeRate(Base):
    __tablename__ = 'exchange_rates'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    source_currency = Column(String)
    target_currency = Column(String)
    conversion_rate = Column(Float)
    timestamp_fetched = Column(DateTime)
    timestamp_recorded = Column(DateTime)

In [35]:
# intialize
Base = declarative_base()
engine = create_engine('sqlite:///currency_exchange.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

In [43]:
# funtion to get data
API_KEY = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'

def get_real_time_fx_data(currency_from, currency_to, amount, precision):
    try:
        # Format the endpoint URL with the appropriate parameters
        endpoint = f"https://api.polygon.io/v1/conversion/{currency_from}/{currency_to}?amount={amount}&precision={precision}&apiKey={API_KEY}"
        
        # Send the request to the API
        response = requests.get(endpoint)
        # If the response status code indicates an error, this will raise an HTTPError.
        response.raise_for_status()
        
        # Parse the response JSON
        data = response.json()
        
        # Check if the response indicates a successful conversion
        if data['status'] == 'success':
            converted_amount = data['converted']
            
            # Return a dictionary with the conversion rate and timestamps
            return {
                'rate_timestamp': datetime.now(),  # Timestamp when the rate was fetched
                'rate': converted_amount,          # The conversion rate obtained
                'entry_timestamp': datetime.now()  # Timestamp when the data is processed
            }
        else:
            print("The conversion request was not successful.")
            return None

    except requests.RequestException as e:
        # Print the error if the request failed
        print(f"Error fetching data: {e}")
        return None

# Example usage of the function
conversion_info = get_real_time_fx_data('EUR', 'GBP', 100, 2)

if conversion_info:
    print(conversion_info)
else:
    print("Failed to fetch conversion information.")

{'rate_timestamp': datetime.datetime(2024, 3, 26, 18, 40, 55, 95034), 'rate': 85.77, 'entry_timestamp': datetime.datetime(2024, 3, 26, 18, 40, 55, 95036)}


In [44]:
#store data
def store_fx_data(rate, from_currency, to_currency):
    conn = sqlite3.connect('fx_data.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO fx_rates (from_currency, to_currency, rate, rate_timestamp, entry_timestamp)VALUES (?, ?, ?, ?, ?)''', (from_currency, to_currency, rate['rate'], rate['rate_timestamp'], rate['entry_timestamp']))
    conn.commit()
    conn.close()

In [48]:
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'

def main():
    session = Session()
    currency_pairs = [('EUR', 'GBP'), ('USD', 'EUR'), ('USD', 'JPY')]
    try:
        for _ in range(10):  # Reduce the range for a quick test
            for from_currency, to_currency in currency_pairs:
                fx_data = fetch_currency_conversion(from_currency, to_currency, 1, 4, api_key)
                if fx_data:
                    rate_record = FXRate(
                        from_currency=from_currency,
                        to_currency=to_currency,
                        rate=fx_data['rate'],
                        rate_timestamp=fx_data['rate_timestamp'],
                        entry_timestamp=fx_data['entry_timestamp']
                    )
                    session.add(rate_record)
                    session.commit()
                    time.sleep(1)  # Rate limit delay
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        session.close()
    
    display_rates()  # Move this outside the try-except block

def display_rates():
    session = Session()
    try:
        rates = session.query(ExchangeRate).limit(10).all()  # Get the latest 10 records
        for rate in rates:
            print(f"ID: {rate.id}, From: {rate.from_currency}, To: {rate.to_currency}, Rate: {rate.rate}, Rate Timestamp: {rate.rate_timestamp}, Entry Timestamp: {rate.entry_timestamp}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        session.close()

if __name__ == '__main__':
    main()

Status Code: 200
Response Body: {"converted":0.8577,"from":"EUR","initialAmount":1,"last":{"ask":0.85775,"bid":0.85769,"exchange":48,"timestamp":1711478776000},"request_id":"9c4bf85394112a5868a824e06a5bf783","status":"success","symbol":"EUR/GBP","to":"GBP"}
Status Code: 200
Response Body: {"converted":0.9231,"from":"USD","initialAmount":1,"last":{"ask":0.92313,"bid":0.92308,"exchange":48,"timestamp":1711478776000},"request_id":"a907ae2b07b684c455a3eece7facda16","status":"success","symbol":"USD/EUR","to":"EUR"}
Status Code: 200
Response Body: {"converted":151.5,"from":"USD","initialAmount":1,"last":{"ask":151.509,"bid":151.5,"exchange":48,"timestamp":1711478776000},"request_id":"30152f6b85da47909495c4918b23c810","status":"success","symbol":"USD/JPY","to":"JPY"}
Status Code: 200
Response Body: {"converted":0.8576,"from":"EUR","initialAmount":1,"last":{"ask":0.8579,"bid":0.8576,"exchange":48,"timestamp":1711478777000},"request_id":"ca303a1aa2386fcb027bce8872da99ce","status":"success","sym

In [32]:
#calculate average fx
import sqlite3

def run_data_collection(api_key):
    # Define the base URL for the Polygon.io Forex API
    base_url = "https://api.polygon.io/v1/last_quote/currencies"

def calculate_average():
    # Connect to the database
    conn = sqlite3.connect('fx_data.db')
    cursor = conn.cursor()
    
    # Execute SQL query to calculate average exchange rate
    query = """
    SELECT from_currency, to_currency, AVG(rate) as avg_rate 
    FROM fx_rates 
    GROUP BY from_currency, to_currency
    """
    cursor.execute(query)
    
    # Fetch all results
    results = cursor.fetchall()
    
    # Iterate over results and print the average rate
    for row in results:
        print(f"From {row[0]} to {row[1]}, Average Rate: {row[2]}")
    
    # Close the database connection
    conn.close()

if __name__ == '__main__':
    api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'
    run_data_collection(api_key)  # Make sure this function is defined
    calculate_average()

From EUR to GBP, Average Rate: 0.8574999582986277
From USD to EUR, Average Rate: 0.9226213868816046
From USD to JPY, Average Rate: 151.40848185231604


In [49]:
# create SQLite DB
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class ExchangeRate(Base):
    __tablename__ = 'exchange_rates'
    id = Column(Integer, primary_key=True, autoincrement=True)
    from_currency = Column(String)
    to_currency = Column(String)
    rate = Column(Float)
    rate_timestamp = Column(DateTime)
    entry_timestamp = Column(DateTime)

# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///currency_exchange.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)


In [52]:
#Create MongoDB
!pip install pymongo
from pymongo import MongoClient
API_KEY = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'
client = MongoClient('localhost', 27017)  # Connect to the MongoDB
db = client['currency_exchange']  # Use or create the database
exchange_rates_collection = db['exchange_rates'] 

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [70]:
# download and store data
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'

# check if 'data' has been defined and has the expected content before proceeding.
currency_pairs = [('EUR', 'GBP'), ('USD', 'EUR'), ('USD', 'JPY')]

# for SQLite
session = Session()

# for MongoDB
client = MongoClient('localhost', 27017)
db = client['currency_exchange']
exchange_rates_collection = db['exchange_rates']

for from_currency, to_currency in currency_pairs:
    response = fetch_currency_conversion(from_currency, to_currency, 100, 2, api_key)
    print(f"API Response for {from_currency} to {to_currency}: {response}")
    if response and response.get('status') == 'success':
        pass  
    else:
        print(f"Failed to fetch data for {from_currency} to {to_currency}")

session.close()

onversion_info = fetch_currency_conversion(from_currency, to_currency, 100, 2, api_key)
print(conversion_info) 

Status Code: 200
Response Body: {"converted":85.77,"from":"EUR","initialAmount":100,"last":{"ask":0.858,"bid":0.8577,"exchange":48,"timestamp":1711481881000},"request_id":"116bd70826e066fcff02adfc324fbabc","status":"success","symbol":"EUR/GBP","to":"GBP"}
API Response for EUR to GBP: None
Failed to fetch data for EUR to GBP
Status Code: 200
Response Body: {"converted":92.31,"from":"USD","initialAmount":100,"last":{"ask":0.9231,"bid":0.92305,"exchange":48,"timestamp":1711481882000},"request_id":"856299e26cb5e5e0f33c9815a48bf6e9","status":"success","symbol":"USD/EUR","to":"EUR"}
API Response for USD to EUR: None
Failed to fetch data for USD to EUR
Status Code: 200
Response Body: {"converted":15152.6,"from":"USD","initialAmount":100,"last":{"ask":151.535,"bid":151.526,"exchange":48,"timestamp":1711481882000},"request_id":"906c4ff6730424b43c6b81dd8e7611b4","status":"success","symbol":"USD/JPY","to":"JPY"}
API Response for USD to JPY: None
Failed to fetch data for USD to JPY
Status Code: 20

In [73]:
import sqlite3
import csv

conn = sqlite3.connect('currency_exchange.db')

cursor = conn.cursor()

cursor.execute("SELECT * FROM exchange_rates")

rows = cursor.fetchall()

csv_file_path = 'exchange_rates.csv'

with open(csv_file_path, 'w', newline='') as file:
    csv_writer = csv.writer(file)
    
    headers = [i[0] for i in cursor.description]
    csv_writer.writerow(headers)

    for row in rows:
        csv_writer.writerow(row)

cursor.close()
conn.close()

print(f'Data exported to {csv_file_path} successfully.')

Data exported to exchange_rates.csv successfully.
